# <h1 align="center"><font color="gree">Mistral AI Agents API Tutorial</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [Obtenha a sua chave de API do Mistral AI](https://console.mistral.ai/api-keys)

* [DataCamp - Mistral AI](https://www.youtube.com/watch?v=VkqYmwIIeOg)

* [Mistral AI Cookbook](https://github.com/mistralai/cookbook/tree/main)

In [3]:
from mistralai import Mistral
from config.settings import MISTRALAI_API_KEY

client = Mistral(api_key=MISTRALAI_API_KEY)

ml_agent = client.beta.agents.create(
    model="mistral-medium-2505",
    name="ml-agent",
    description="Asistente de Machine learning",
    instructions="Você é um especialista em Machine learning. Dá conselhos práticos e ações.",
)



In [6]:
print(f"Agent created: {ml_agent.id}")

Agent created: ag_0197f50e699e73fcb8ba6f1ae3eb8002


In [7]:
response = client.beta.conversations.start(
    agent_id=ml_agent.id,
    inputs="Devo usar Random Forest ou XGBoost para um dataset de 5000 amostras? Responda em uma sentença apenas.",
)


In [8]:
type(response)

mistralai.models.conversationresponse.ConversationResponse

In [9]:
response.outputs

[MessageOutputEntry(content='Para um dataset de 5000 amostras, XGBoost geralmente oferece melhor desempenho e eficiência.', object='entry', type='message.output', created_at=datetime.datetime(2025, 7, 10, 16, 3, 36, 101011, tzinfo=TzInfo(UTC)), completed_at=datetime.datetime(2025, 7, 10, 16, 3, 36, 406929, tzinfo=TzInfo(UTC)), id='msg_0197f513f82476df8bd7cb082bce3698', agent_id='ag_0197f50e699e73fcb8ba6f1ae3eb8002', model='mistral-medium-2505', role='assistant')]

In [10]:
print(response.outputs[0].content)

Para um dataset de 5000 amostras, XGBoost geralmente oferece melhor desempenho e eficiência.
